In [1]:
# @title Installations
# The %%capture command hides the installation output.
%%capture
!pip install langchain
!pip install lark
!pip install openai
!pip install chromadb
!pip install langchain_openai
!pip install langchain_chroma
!pip install langchain_community
!pip install langchain_community



In [17]:
# @title Imports
from langchain_core.documents import Document
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.chat_models import ChatOpenAI
import os
from google.colab import userdata


In [9]:

# Cell 3: Docs
docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]


In [18]:
# @title Environment Setup
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [19]:
# @title Chroma DB and OpenAI Embeddings
vectordb = Chroma.from_documents(docs, OpenAIEmbeddings())

In [20]:
# @title Metadata Field Information
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating",
        description="A 1-10 rating for the movie",
        type="float",
    ),
]


In [21]:
document_content_description = "Brief summary of a movie"

retriever = SelfQueryRetriever.from_llm(
    ChatOpenAI(temperature=0),
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True,
)

retriever.invoke("I want to watch a movie rated higher than 8.5")

/tmp/ipython-input-3551695438.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(temperature=0),


[Document(id='80fd3e07-32a6-46f3-a759-7fdca250b9ed', metadata={'rating': 9.9, 'genre': 'thriller', 'year': 1979, 'director': 'Andrei Tarkovsky'}, page_content='Three men walk into the Zone, three men walk out of the Zone'),
 Document(id='68f0ac93-f9e6-4672-b3b3-0a80481e4abd', metadata={'rating': 8.6, 'year': 2006, 'director': 'Satoshi Kon'}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

In [22]:
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(id='7b0955d1-8c16-4078-8c23-ce426d2e1bc7', metadata={'rating': 8.3, 'year': 2019, 'director': 'Greta Gerwig'}, page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them')]

In [33]:
retriever.invoke("science fiction films")

[Document(id='e4429e29-fec7-4fba-b730-e6e3899929de', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}, page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose'),
 Document(id='68f0ac93-f9e6-4672-b3b3-0a80481e4abd', metadata={'year': 2006, 'rating': 8.6, 'director': 'Satoshi Kon'}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(id='6a01dd1d-fd90-4bbe-8ca2-6117178edbe0', metadata={'rating': 8.2, 'year': 2010, 'director': 'Christopher Nolan'}, page_content='Leo DiCaprio gets lost in a dream within a dream within a ...'),
 Document(id='e9eadd56-bccf-4430-92ce-07d9eac1848b', metadata={'year': 1995, 'genre': 'animated'}, page_content='Toys come alive and have a blast doing so')]